# Linea de Muerte Z

##  1. Big Picture

* Preprocesamiento
   * Se quitan del datataset los *envenenados* atributos  mprestamos_personales y cprestamos_personales
   * Se agregan lags y delta_lags de orden 1 y 2
* Modelado no se optimizan hiperarámetros
* Produccion
   * Entrenamieento final
      * Se entrena en {202101, 202102, 202103, 202104}
      * Se hace un conservador undersampling de **0.50** de los "CONTINUA"
      * POS = {"BAJA+1", "BAJA+2"}
      * librería  *zLightGBM*
         * **100** canaritos se agregan al comienzo del dataset, porque ahora hay mas datos
         * gradient_bound se deja en su default de  **0.1**
   * Clasificacion
      * Se corta en 11000  envios

---
Resultados :
* ganancia de 391.085 M en el Private Leaderboard ( 11.000 en el Public )
* utiliza 17 GB de memoria RAM
* corre en 30 minutos

## Inicializacion

In [1]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-11-16 17:34:21 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,658618,35.2,1454640,77.7,1189808,63.6
Vcells,1222364,9.4,8388608,64.0,1975153,15.1


In [2]:
PARAM <- list()
PARAM$experimento <- "Definitivo1"
PARAM$semilla_primigenia <- 484643

In [3]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

## Preprocesamiento

### Generacion de la clase_ternaria

In [4]:
Sys.time()
require( "data.table" )

# leo el dataset
dataset <- fread("~/buckets/b1/datasets/competencia_02_crudo.csv" )

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

[1] "2025-11-16 17:34:21 UTC"

Loading required package: data.table



,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,723537,38.7,1454640,77.7,1454640,77.7
Vcells,113421211,865.4,211929521,1616.9,211593776,1614.4


[1] "2025-11-16 17:34:25 UTC"

In [5]:
meses_a_eliminar <- c(202006, 201910)

dataset <- dataset[ !(foto_mes %in% meses_a_eliminar) ]

### Eliminacion de Features

La auténtica  **salsa mágica**  de este script es la eliminación de estos dos atributos del dataset ya que tran problemas con el Data Drifting
* mprestamos_personales
* cprestamos_personales

el problema con estos campos se detectó manualmente mediante un analisis exploratorio de datos y análisis de corridas de LightGBM en meses previos.

In [6]:
dataset[, mprestamos_personales := NULL ]
dataset[, cprestamos_personales := NULL ]

Sys.time()

[1] "2025-11-16 17:34:25 UTC"

## Feature Engineer Intra Mes

In [7]:
# el mes 1,2, ..12 , podria servir para detectar estacionalidad
dataset[, kmes := foto_mes %% 100]

# creo un ctr_quarter que tenga en cuenta cuando
# los clientes hace 3 menos meses que estan
# ya que seria injusto considerar las transacciones medidas en menor tiempo
dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]
dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5.0]
dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2.0]
dataset[cliente_antiguedad == 3, ctrx_quarter_normalizado := ctrx_quarter * 1.2]

# variable extraida de una tesis de maestria de Irlanda, se perdió el link
dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

Sys.time()

[1] "2025-11-16 17:34:25 UTC"

### Feature Engineering Historico

A partir del dataset crudo, se genera el campo  clase_ternaria

In [8]:
if( !require("Rcpp")) install.packages("Rcpp", repos = "http://cran.us.r-project.org")
require("Rcpp")

Loading required package: Rcpp



In [9]:
# se calculan para los 6 meses previos el minimo, maximo y
#  tendencia calculada con cuadrados minimos
# la formula de calculo de la tendencia puede verse en
#  https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
# para la maxíma velocidad esta funcion esta escrita en lenguaje C,
# y no en la porqueria de R o Python

cppFunction("NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde )
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) )
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      {
        xsum  += x[h] ;
        ysum  += y[h] ;
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ;
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}")

In [10]:
# calcula la tendencia de las variables cols de los ultimos 6 meses
# la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
# La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas <- function(
    dataset, cols, ventana = 6, tendencia = TRUE,
    minimo = TRUE, maximo = TRUE, promedio = TRUE,
    ratioavg = FALSE, ratiomax = FALSE) {
  gc(verbose= FALSE)
  # Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion <- ventana

  last <- nrow(dataset)

  # creo el vector_desde que indica cada ventana
  # de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids <- dataset[ , numero_de_cliente ]

  vector_desde <- seq(
    -ventana_regresion + 2,
    nrow(dataset) - ventana_regresion + 1
  )

  vector_desde[1:ventana_regresion] <- 1

  for (i in 2:last) {
    if (vector_ids[i - 1] != vector_ids[i]) {
      vector_desde[i] <- i
    }
  }
  for (i in 2:last) {
    if (vector_desde[i] < vector_desde[i - 1]) {
      vector_desde[i] <- vector_desde[i - 1]
    }
  }

  for (campo in cols) {
    nueva_col <- fhistC(dataset[, get(campo)], vector_desde)

    if (tendencia) {
      dataset[, paste0(campo, "_tend", ventana) :=
        nueva_col[(0 * last + 1):(1 * last)]]
    }

    if (minimo) {
      dataset[, paste0(campo, "_min", ventana) :=
        nueva_col[(1 * last + 1):(2 * last)]]
    }

    if (maximo) {
      dataset[, paste0(campo, "_max", ventana) :=
        nueva_col[(2 * last + 1):(3 * last)]]
    }

    if (promedio) {
      dataset[, paste0(campo, "_avg", ventana) :=
        nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratioavg) {
      dataset[, paste0(campo, "_ratioavg", ventana) :=
        get(campo) / nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratiomax) {
      dataset[, paste0(campo, "_ratiomax", ventana) :=
        get(campo) / nueva_col[(2 * last + 1):(3 * last)]]
    }
  }
}

In [11]:
# Feature Engineering Historico
# Creacion de LAGs
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
  colnames(dataset),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
))

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
  paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 2
dataset[,
  paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
  dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
  dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}

Sys.time()

[1] "2025-11-16 17:35:49 UTC"

In [12]:
# parametros de Feature Engineering Historico de Tendencias
PARAM$FE_hist$Tendencias$run <- TRUE
PARAM$FE_hist$Tendencias$ventana <- 6
PARAM$FE_hist$Tendencias$tendencia <- TRUE
PARAM$FE_hist$Tendencias$minimo <- TRUE
PARAM$FE_hist$Tendencias$maximo <- TRUE
PARAM$FE_hist$Tendencias$promedio <- TRUE
PARAM$FE_hist$Tendencias$ratioavg <- TRUE
PARAM$FE_hist$Tendencias$ratiomax <- TRUE

In [13]:
# aqui se agregan las tendencias de los ultimos 6 meses

cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    TendenciaYmuchomas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}

ncol(dataset)
Sys.time()

[1] 1178

[1] "2025-11-16 17:36:23 UTC"

## Modelado

### Optimizacion de Hipeparámetros

Este script no hace optimización de hiperparámetros
<br> **No** se llama a una Bayesian Optimization, ese paso se saltea.
<br> No hace falta hacer particiones <train, validate, test>,  tampoco se hace un k-fold cross validation

## Produccion

Las decisiones que se toman para la construccion del modelo final son:
* Los positvos son  POS={"BAJA+1", "BAJA+2"}, esta es una meticulosa decisión.
* Se entrena en los cuatro meses  {202101, 202102, 202103, 202104}, esta es una meticulosa decisión.
* Se hace un muy consevador undersampling del **0.50** = 50% de la clase mayoritaria (los "CONTINUA" )
* Obviamente los datos donde se aplica el modelo es el mes  {202106}
* Por experimentos en meses anteriores, se decide cortar en los 11000 registros con mayor probabildiad de POS={"BAJA+1", "BAJA+2"}, , esta es una *enorme* decisión.
* No se optmizan los hiperparámetros de LightGBM, sino que se llama a la libreria *zLightGBM*
* Para *zLightGBM*  se crean **100** canaritos, esta es una decisión enorme !

In [14]:
# training y future
Sys.time()

PARAM$train_final$meses <- c(201901,201902,201902,201903,201904,201905,
                             201906,201907,201908,201909,201911,201912,
                             202001,202002,202003,202004,202005,202007,202008,
                             202009,202010,202012,202101, 202102, 202103, 202104,202105,202106)
PARAM$train_final$undersampling <- 0.05

PARAM$future <- c(202108)

[1] "2025-11-16 17:36:23 UTC"

### Final Training Strategy

Se entrena en los cuatro meses {202101, 202102, 202103, 202104}
<br>Se hace un conservador undersampling del 50% de la clase mayoritaria (los "CONTINUA" )

In [15]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$meses]

#### Registros cambio las proporciones de POS/NEG

In [16]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [17]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

### Final Model

In [18]:
# utilizo  zLightGBM  la nueva libreria
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")
Sys.time()

Loading required package: zlightgbm



[1] "2025-11-16 17:36:25 UTC"

Un hiperparámetro de  *zLightGBM* es la cantidad de canaritos, en este caso se decidió (con tests realizados en {202101, 202102} fijarlo en 100
<br> esta es una importante decisión que debe tomarse
<br> en la version actual de *zLightGBM* los canaritos deben agregarse por fuera al dataset, y mandatoriamente deben ser las primeras columnas del mismo
<br> durante noviembre, si hay suerte, se liberará una versiónn que lo haga automáticamente internamente en el código C++ de la librería

In [19]:
# canaritos
PARAM$qcanaritos <- 5

cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)

for( i in seq(PARAM$qcanaritos) ){
  dataset_train_final[, paste0("canarito_",i) := runif( filas) ]
}

# las columnas canaritos mandatoriamente van al comienzo del dataset
cols_canaritos <- copy( setdiff( colnames(dataset_train_final), cols0 ) )
setcolorder( dataset_train_final, c( cols_canaritos, cols0 ) )

Sys.time()

[1] "2025-11-16 17:36:25 UTC"

In [20]:
# los campos que se van a utilizar

campos_buenos <- setdiff(
  colnames(dataset_train_final),
  c("clase_ternaria", "clase01", "training")
)

In [21]:
# dejo los datos en el formato que necesita LightGBM

dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 57101 columnas 1182 


[1] "2025-11-16 17:36:26 UTC"

Nuevos hiperparámetros en  *zLightGBM*
* canaritos , entero >=0,  cantidad de atributos del dataset que se consideran como canaritos, mandatoriamente en la version actual deben estar al comienzo.
* gradient_bound , numero real, >=0  cota que se pone a los scores de las hojas de los arboles, es un learning_rate adaptativo

En el caso que  canaritos==0  y  gradient_bound==0  entonces  xLightGBM se comporta exactamente igual a  LightGBM

Tener en cuenta que el hiperparámetro  min_data_in_leaf se está dejando en el valor de 20 que es el default de LightGBM
<br> realmente valdria la pena experimentar con distintos valores de  min_data_in_leaf, hay potencial de mejora !

In [22]:
# definicion de parametros, los viejos y los nuevos

PARAM$lgbm <-  list(
  boosting= "gbdt",
  objective= "binary",
  metric= "custom",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE,
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_bin= 31L,
  min_data_in_leaf= 2000L,  #este ya es el valor default de LightGBM

  num_iterations= 9999L, # dejo libre la cantidad de arboles, zLightGBM se detiene solo
  num_leaves= 999L, # dejo libre la cantidad de hojas, zLightGBM sabe cuando no hacer un split
  learning_rate= 1.0,  # se lo deja en 1.0 para que si el score esta por debajo de gradient_bound no se lo escale
    
  feature_fraction= 0.50, # un valor equilibrado, habra que probar alternativas ...
    
  canaritos= PARAM$qcanaritos, # fundamental en zLightGBM, aqui esta el control del overfitting
  gradient_bound= 0.01  # default de zLightGBM
)

Sys.time()

[1] "2025-11-16 17:36:26 UTC"

####  Entrenamiento del modelo

Aqui se ejecuta  *zLightGBM*
<br> no se utilizan hiperparámetros optimos para controlar el overfitting
<br> zLightGBM  sabe cuando no hacer un split
<br> si al hacer el n-simo arbol, no puede hacer el split de la raiz, detiene el crecimiento del ensemble y termina
<br>
<br> claramente el  min_data_in_leaf=20 que es el default de LightGBM esta jugando un papel importante

In [23]:
# entreno el modelo

modelo_final <- lgb.train(
  data= dtrain_final,
  param= PARAM$lgbm
)

Sys.time()

[1] "2025-11-16 17:45:41 UTC"

In [24]:
# grabo el modelo generado, esto pude ser levantado por LighGBM en cualquier maquina
lgb.save(modelo_final, file="zmodelo.txt")

# grabo un dataset que tiene el detalle de los arboles de LightGBM
tb_arboles <- lgb.model.dt.tree(modelo_final)
fwrite(tb_arboles, file="tb_arboles.txt", sep="\t")

cat("cantidad arbolitos=", tb_arboles[, max(tree_index)+1],"\n" )
cat("summary de las hojas de los arboles")
summary( tb_arboles[, list(hojas=max(leaf_index, na.rm=TRUE)+1), tree_index][,hojas])

Sys.time()

cantidad arbolitos= 2093 
summary de las hojas de los arboles

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   2.00   20.00   22.00   20.92   23.00   27.00 

[1] "2025-11-16 17:45:50 UTC"

In [26]:
dfuture

Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,⋯,ccomisiones_mantenimiento_avg6,ccomisiones_mantenimiento_ratioavg6,ccomisiones_mantenimiento_ratiomax6,mcomisiones_mantenimiento_tend6,mcomisiones_mantenimiento_min6,canarito_1,canarito_2,canarito_3,canarito_4,canarito_5
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


### Scoring

Se hace el predict() del modelo en los datos del futuro

In [25]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes %in% PARAM$future]

# penosamente, en la versión actual de zLightGBM  los campos canaritos
#  aunque no se utilizan para nada, también deben estar en el dataset donde se hace el predict()
filas <- nrow(dfuture)

for( i in seq(PARAM$qcanaritos) ){
  dfuture[, paste0("canarito_",i) := runif( filas) ]
}

prediccion <- predict(
  modelo_final,
  data.matrix(dfuture[, campos_buenos, with= FALSE]),
)

ERROR: Error in if (length(preds)%%num_row != 0L) {: missing value where TRUE/FALSE needed


In [ ]:
# tabla de prediccion, puede ser util para futuros ensembles
#  ya que le modelo ganador va a ser un ensemble de LightGBMs

tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]
tb_prediccion[, prob := prediccion ]

# grabo las probabilidad del modelo
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

### Clasificacion

Se tomó la decisión de enviar a los 11000 registros con mayor probabilidad de POS={"BAJA+1","BAJA+"}
<br> esto se determinó en forma artesanal analizando meses anterior
<br> esta es una muy importante decisión 

In [ ]:
# genero archivos con los  "envios" mejores
dir.create("kaggle", showWarnings=FALSE)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

envios <- 11000
tb_prediccion[, Predicted := 0L] # seteo inicial a 0
tb_prediccion[1:envios, Predicted := 1L] # marco los primeros

archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

# grabo el archivo
fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
  file= archivo_kaggle,
  sep= ","
)

In [ ]:
clientes_predichos <- tb_prediccion[
    Predicted == 1,                          # Condición de fila (i)
    list(numero_de_cliente)                  # Columnas a seleccionar (j)
]

In [ ]:
archivo_kaggle_testeo <- paste0("./kaggle/KA_DEF", PARAM$experimento, "_", envios, ".csv")

# grabo el archivo
fwrite(clientes_predichos[, list(numero_de_cliente)],
  file= archivo_kaggle-testeo,
  sep= ","
)

### Subida a Kaggle

In [ ]:
Sys.time()